In [118]:
from faker import Faker
# Conexion
import psycopg2
import uuid

# Configuración de la conexion
conexion = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="contraseña"
)

# Consulta para obtener las tablas de un esquema
esquema = 'recruitment'
tablas_db = []
consulta = f"""
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = '{esquema}' AND table_type = 'BASE TABLE';
"""

# Ejecutar la consulta
with conexion.cursor() as cursor:
    cursor.execute(consulta)
    tablas = cursor.fetchall()
    tablas_db = tablas

In [119]:

# def recorrer_tabla(tabla):
#     # Consulta para obtener las columnas de una tabla
#     consulta = f"""
#     SELECT column_name, data_type
#     FROM information_schema.columns
#     WHERE table_schema = %s AND table_name = %s;
#     """

#     # Ejecutar la consulta
#     with conexion.cursor() as cursor:
#         cursor.execute(consulta, (esquema, tabla))
#         columnas = cursor.fetchall()

#     # Cerrar la conexión

#     print(tabla)
#     # Mostrar las columnas
#     for columna in columnas:
#         print(f"Columna: {columna[0]}, Tipo de dato: {columna[1]}")

In [120]:
# for tabla in tablas_db:
#     recorrer_tabla(tabla)

In [121]:
# conexion.close()

In [122]:
# for tabla in tablas_db:
#     print(tabla[0])

In [123]:
# Instancia Faker
fake = Faker()
import random

In [124]:
def generar_datos_applicant_profile(num_registros):
    datos = []
    for _ in range(num_registros):
        nombre = fake.name()
        email = fake.email()
        id_usuario = str(uuid.uuid4())
        
        # Aquí puedes añadir otras columnas según el diseño de tu tabla
        datos.append((nombre, email, id_usuario))
    return datos

In [125]:
registros = generar_datos_applicant_profile(20)

In [126]:
registros

[('Laura Anderson',
  'jmcknight@example.org',
  '2b373a44-2c12-4369-ba47-9b4d04d139aa'),
 ('Luis Banks',
  'youngbrandon@example.org',
  'e024f310-a38e-47a3-905c-8fae90fd36f2'),
 ('Anna Richards',
  'dsanchez@example.com',
  'aa95ae5d-107d-406a-89cd-08d61564b8ac'),
 ('Steven Frank',
  'fwillis@example.net',
  'ea40ad88-f970-4aab-be4e-cd6f72ecca90'),
 ('Linda Day',
  'frederickjames@example.org',
  '29b66547-c346-42f1-807a-8cf09b82b050'),
 ('Kristopher Robinson',
  'edward83@example.net',
  '8ade34e2-c40f-4dbf-82b7-4a987a740f73'),
 ('George Morgan',
  'doughertymaurice@example.org',
  '2dda637e-2dbf-46a7-b68d-50f4566622f9'),
 ('Vicki Irwin',
  'gabrielapatterson@example.com',
  '859f7351-7742-4ba8-b369-89ee4ed0e00f'),
 ('Virginia Baker',
  'pennyanderson@example.com',
  'c6c2f504-de8c-46ae-8db2-c1b20c042216'),
 ('Edward Serrano',
  'scottdaniel@example.net',
  '9c1a5ba8-5439-4f17-a0c5-3186334f1651'),
 ('Philip Smith',
  'samantha31@example.org',
  '9ccd4be1-527a-492c-9288-1ab2ac40c61a'

In [127]:
# Insertar los datos generados en la base de datos
def insertar_datos_applicant_profile(datos):
    with conexion.cursor() as cursor:
        # Consulta SQL para insertar registros
        
        consulta = """
        INSERT INTO recruitment.user (name, email, user_id)
        VALUES (%s, %s, %s)
        """
        # Insertar múltiples registros
        cursor.executemany(consulta, datos)
        conexion.commit()  # Confirmar la transacción

In [128]:
insertar_datos_applicant_profile(registros)

In [151]:
import psycopg2

def insertar_user_ids_no_existentes():
    try:
        # Conectar a la base de datos
        with psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="contraseña"
        ) as conexion:
            with conexion.cursor() as cursor:
                # Leer los 'user_id' existentes en la tabla 'user'
                consulta_user_ids = """
                SELECT user_id FROM recruitment.user
                """
                cursor.execute(consulta_user_ids)
                user_ids_existentes = set(row[0] for row in cursor.fetchall())  # Almacenar los user_id en un set

                # Leer los 'user_id' que ya existen en la tabla 'applicant_profile'
                consulta_applicant_profile = """
                SELECT user_id FROM recruitment.applicant_profile
                """
                cursor.execute(consulta_applicant_profile)
                user_ids_en_profile = set(row[0] for row in cursor.fetchall())  # Almacenar los user_id en applicant_profile

                # Identificar los 'user_id' que están en 'user' pero no en 'applicant_profile'
                user_ids_a_insertar = user_ids_existentes - user_ids_en_profile

                if user_ids_a_insertar:
                    # Insertar los 'user_id' que faltan en la tabla 'applicant_profile'
                    consulta_insertar = """
                    INSERT INTO recruitment.applicant_profile (user_id)
                    VALUES (%s)
                    """
                    cursor.executemany(consulta_insertar, [(user_id,) for user_id in user_ids_a_insertar])
                    conexion.commit()  # Confirmar la transacción

                    print(f"{len(user_ids_a_insertar)} user_ids insertados correctamente en 'applicant_profile'.")
                else:
                    print("No hay nuevos user_ids para insertar en 'applicant_profile'.")
                    return True

    except Exception as e:
        # Imprimir el error en caso de que ocurra una excepción
        print(f"Ocurrió un error: {e}")
        # Si ocurre un error, hacer rollback de cualquier cambio no confirmado
        if 'conexion' in locals():
            conexion.rollback()


In [ ]:
while (True):
    if(insertar_user_ids_no_existentes()):
        break

No hay nuevos user_ids para insertar en 'applicant_profile'.
